# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
try:
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])
except OSError as e:
    print(e)

USER = 'plggarstka'

[WinError 183] Cannot create a file when that file already exists: 'E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh'


Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [9]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=KeyType.RSA,
                      install_key=True)

2018-08-03 23:05:00 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [29]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_qj', install_key=False, disable_sshd=False)

In [5]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)
save_environment('.idact-env')

In [6]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

In [7]:
node.run('whoami')

'plggarstka'

In [8]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [9]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=1,
                               memory_per_node=bitmath.MiB(100),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'ccbmc7'
                               })

In [10]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)])

In [11]:
nodes.wait()
nodes

Nodes([Node(p0652:43604, 2018-08-03 22:27:03.876621+00:00),Node(p0666:50771, 2018-08-03 22:27:03.876621+00:00)])

## Run commands

In [12]:
nodes[0].run('whoami')

'plggarstka'

In [13]:
nodes[0].run('hostname')

'p0652'

In [14]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          12429616 plgrid-te     wrap plggarst  R       0:20      2 p[0652,0666]'

In [15]:
nodes[1].run('hostname')

'p0666'

## Tunnel

In [16]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [17]:
tunnel

MultiHopTunnel(9000:10000)

In [18]:
tunnel.close()

## Deploy notebook

One-time config step (cluster-specific):

In [20]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment('.idact-env')

To run Jupyter Notebook on the cluster:

In [21]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

2018-08-04 00:10:27 WARNING: Exception: Cannot run task., retry 1/5.



Fatal error: run() received nonzero return code 1 while executing!

Requested: cat '/net/people/plggarstka/.idact/runtime/ihLaYYaWVZGJNP260915CbhxCxIGD7oj/nbserver-*.json' > /dev/null
Executed: /bin/bash -l -c "cat '/net/people/plggarstka/.idact/runtime/ihLaYYaWVZGJNP260915CbhxCxIGD7oj/nbserver-*.json' > /dev/null"

=============================== Standard output ===============================

cat: /net/people/plggarstka/.idact/runtime/ihLaYYaWVZGJNP260915CbhxCxIGD7oj/nbserver-*.json: No such file or directory


Aborting.


JupyterDeployment(8080 -> Node(p0652:43604, 2018-08-03 22:27:03.876621+00:00))

In [22]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [23]:
nb.open_in_browser()

In [24]:
nb.cancel()

## Close

In [25]:
nodes.running()

True

In [26]:
nodes.cancel()

In [27]:
nodes.running()

False

In [28]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)'